In [1]:
import os
PATH = os.getenv("HOME")
os.chdir(f"{PATH}/watttime-python-client-aer-algo")

import pandas as pd
import evaluation.eval_framework as evu

import seaborn as sns
import evaluation.metrics as m
import datetime as dt

import data.s3 as s3u

import warnings
warnings.filterwarnings("ignore")

In [2]:
s3 = s3u.s3_utils()

In [12]:
original_regions = [
    "SPP_TX",
    "ERCOT_EASTTX",
    "FPL",
    "SOCO",
    "PJM_CHICAGO",
    "LDWP",
    "PJM_DC",
    "NYISO_NYC",
]

In [13]:
region = original_regions[2]
dat = s3.load_csvdataframe(f"results/analysis_requery_20241212_v3_{region}.csv")

In [19]:
del dat

In [7]:
dat["uuid"].nunique()

4812

In [11]:
del dat

In [3]:
original_regions = [
    "SPP_TX",
    "ERCOT_EASTTX",
    "FPL",
    "SOCO",
    "PJM_CHICAGO",
    "LDWP",
    "PJM_DC",
    "NYISO_NYC",
]

random_regions = [
    'PACE',
    'PNM',
    'MISO_INDIANAPOLIS',
    'WALC',
    'ERCOT_AUSTIN',
    'SPP_KANSAS',
    'ISONE_VT',
    'SPP_SIOUX',
    'SC'
]

regions = original_regions+random_regions

dfs = []
for region in regions:
    try:
        dat = s3.load_csvdataframe(f"results/analysis_requery_20241212_v3_{region}.csv")
        dfs.append(dat)
    except:
        print(region)
        continue

FPL


In [8]:
df = pd.concat(dfs)

In [9]:
df.shape

(16428, 11)

In [18]:
df[["user_type","region","requery_increment"]].drop_duplicates().shape

(218, 3)

In [25]:
df["region"].nunique()

2

In [21]:
increments = [5,15,30,60,90,120,180,240]

In [27]:
(len(increments)+3)*10*2

220

In [11]:
df2 = pd.concat(dfs)

In [12]:
df2

,index,distinct_dates,user_type,session_start_time,session_end_time,initial_charge,total_seconds_to_95,full_charge_time,length_of_session_in_seconds,charged_kWh_actual,...,total_intervals_plugged_in,charge_MWh_needed,charged_MWh_actual,MWh_fraction,usage_time_required_minutes,contiguous_block,charge_per_interval,sanitize_intervals_plugged_in,sanitize_time_needed,uuid
0,13,2024-02-15,r22.7375_tc71_avglc29236_sdlc7968_contFalse,2024-02-15 09:07:03,2024-02-15 15:07:03,0.563397,4345.93915,2024-02-15 10:19:28.939150210,21600.0,27.448831,...,72.0,0.027449,0.027449,0.001895,73.0,False,NaN,72,75,r22.7375_tc71_avglc29236_sdlc7968_contFalse|20...


In [8]:
check = df[["user_type","requery_increment"]].drop_duplicates().groupby("requery_increment")["region"].value_counts().reset_index()

In [13]:

import os
PATH = os.getenv("HOME")
os.chdir(f"{PATH}/watttime-python-client-aer-algo")

import pandas as pd
import evaluation.eval_framework as evu
from datetime import datetime

import seaborn as sns
import evaluation.metrics as m
from datetime import timedelta

import random
import math
from watttime import WattTimeForecast, WattTimeHistorical
import data.s3 as s3u
import importlib
import watttime.api as wt

import warnings
warnings.filterwarnings("ignore")

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

s3 = s3u.s3_utils()

sample_size = 30000

df_req = s3.load_csvdataframe("requery_data/20241203_1k_synth_users_96_days.csv")

def sanitize_time_needed(x,y):
    return int(math.ceil(min(x, y) / 300.0) * 5)

def sanitize_total_intervals(x):
    return math.ceil(x)

def load_forecast_file(region):
    full_forecast = s3.load_parquetdataframe(
    f"complete_2024_forecast_history/{region}.parquet"
    ).drop_duplicates()

    full_forecast["point_time"] = pd.to_datetime(
    full_forecast["point_time"], utc=True
    )

    return full_forecast

def load_history_file(region):
    return s3.load_parquetdataframe(f"complete_2024_actual_history/{region}.parquet").drop_duplicates()

def prepare_set_of_historic_actuals(
        full_history,
        start_time,
        end_time
        ):

        moer_list = full_history.loc[
                (full_history["point_time"] >= start_time - timedelta(minutes=5)) &
                (full_history["point_time"] <= end_time - timedelta(minutes=5))
                ]
        return moer_list

def prepare_set_of_forecasts(
                forecasts,
                start_time,
                end_time,
                increment
    ):
        inc_times = pd.date_range(
            start_time,
            end_time,
            freq=timedelta(minutes=increment),
        ).tolist()

        moer_list = [
            forecasts.loc[
            forecasts["generated_at"] == timestamp].sort_values(by=["point_time"], ascending=True)
            for timestamp in inc_times
        ]

        return moer_list

def get_recalculating_optimizer_results(
    region: str,
    moer_list: pd.DataFrame,
    start_time: datetime,
    end_time: datetime,
    usage_power_kw,
    time_needed,
    increment,
    charge_per_interval = None,
):
    
    if charge_per_interval is None:
        wt_opt_rc = wt.RecalculatingWattTimeOptimizer(
            region=region,
            watttime_username=username,
            watttime_password=password,
            usage_time_required_minutes=time_needed,
            usage_power_kw=usage_power_kw,
            optimization_method="auto"
            )
    else:
        wt_opt_rc = wt.RecalculatingWattTimeOptimizerWithContiguity(
            username,
            password,
            region,
            usage_time_required_minutes=time_needed,
            usage_power_kw=usage_power_kw,
            optimization_method='auto',
            charge_per_interval=charge_per_interval,
        )

    new_start_time = start_time
    for fcst_data in moer_list:
        new_start_time = pd.Timestamp(fcst_data["point_time"].min())
        wt_opt_rc.get_new_schedule(
                new_start_time=new_start_time,
                new_end_time=end_time,
                curr_fcst_data=fcst_data
                )
    print("combining schedules")
    usage_plan = wt_opt_rc.get_combined_schedule(end_time=end_time)
    usage_plan["requery_increment"] = increment

    return usage_plan

df_req["sanitize_intervals_plugged_in"] = df_req.apply(lambda x: sanitize_total_intervals(x.total_intervals_plugged_in), axis=1)
df_req["sanitize_time_needed"] = df_req.apply(lambda x: sanitize_time_needed(x.total_seconds_to_95, x.length_of_session_in_seconds), axis=1)

synth_data = df_req.sample(sample_size, random_state=42).copy()
synth_data.session_start_time = pd.to_datetime(synth_data.session_start_time)
synth_data.session_end_time = pd.to_datetime(synth_data.session_end_time)

original_regions = [
    "SPP_TX",
    "ERCOT_EASTTX",
    "FPL",
    "SOCO",
    "PJM_CHICAGO",
    "LDWP",
    "PJM_DC",
    "NYISO_NYC",
]

random_regions = [
    'PACE',
    'PNM',
    'MISO_INDIANAPOLIS',
    'WALC',
    'ERCOT_AUSTIN',
    'SPP_KANSAS',
    'ISONE_VT',
    'SPP_SIOUX',
    'SC'
]

regions = original_regions+random_regions

In [14]:
synth_data.shape[0]

30000

In [45]:
synth_data.head()

,index,distinct_dates,user_type,session_start_time,session_end_time,initial_charge,total_seconds_to_95,full_charge_time,length_of_session_in_seconds,charged_kWh_actual,...,power_output_rate,total_intervals_plugged_in,charge_MWh_needed,charged_MWh_actual,MWh_fraction,usage_time_required_minutes,contiguous_block,charge_per_interval,sanitize_intervals_plugged_in,sanitize_time_needed
44901,51,2024-06-28,r24.819999999999997_tc90_avglc27711_sdlc7448_c...,2024-06-28 06:14:05,2024-06-28 09:14:05,0.584669,4769.020903,2024-06-28 07:33:34.020903308,10800.0,32.879750,...,24.8200,36.0,0.032880,0.032880,0.002068,80.0,False,NaN,36,80
3104,25,2024-03-31,r29.070000000000004_tc104_avglc23684_sdlc7351_...,2024-03-31 12:34:05,2024-03-31 15:34:05,0.539796,5283.121347,2024-03-31 14:02:08.121346858,10800.0,42.661205,...,29.0700,36.0,0.042661,0.042661,0.002423,89.0,False,NaN,36,90
23444,13,2024-02-15,r22.7375_tc71_avglc29236_sdlc7968_contFalse,2024-02-15 09:07:03,2024-02-15 15:07:03,0.563397,4345.939150,2024-02-15 10:19:28.939150210,21600.0,27.448831,...,22.7375,72.0,0.027449,0.027449,0.001895,73.0,False,NaN,72,75
18746,48,2024-06-19,r30.7275_tc103_avglc23319_sdlc6830_contFalse,2024-06-19 15:44:25,2024-06-19 18:44:25,0.514997,5249.338338,2024-06-19 17:11:54.338337763,10800.0,44.805290,...,30.7275,36.0,0.044805,0.044805,0.002561,88.0,False,NaN,36,90
52588,48,2024-06-19,r26.945_tc87_avglc24445_sdlc6817_contFalse,2024-06-19 17:38:34,2024-06-20 05:38:34,0.552190,4624.015896,2024-06-19 18:55:38.015895520,43200.0,34.609475,...,26.9450,144.0,0.034609,0.034609,0.002245,78.0,False,NaN,144,80


In [51]:
region = "ERCOT_EASTTX"

In [52]:
    full_forecast = load_forecast_file(region)
    full_history = load_history_file(region)

In [53]:
for i in range(0,synth_data.shape[0]):
    loc_num = i
    time_zone = evu.get_timezone_from_dict(region)                    
    start_time_utc = pd.Timestamp(evu.convert_to_utc(synth_data.iloc[loc_num]['session_start_time'].round('5min') , time_zone))
    end_time_utc = pd.Timestamp(evu.convert_to_utc(synth_data.iloc[loc_num]['session_end_time'].round('5min'), time_zone))
    time_needed = synth_data.iloc[loc_num]["sanitize_time_needed"]
    total_intervals_plugged_in = synth_data.iloc[loc_num]["sanitize_intervals_plugged_in"]
    usage_power_kw = float(synth_data.iloc[loc_num]["power_output_rate"])
    user_type = synth_data.iloc[loc_num]["user_type"]
    optimization_method = "auto"
    uuid = user_type + "|" + str(start_time_utc) + "|" + region
    
    results_dfs = []
    for increment in increments:
            moer_list = prepare_set_of_forecasts(
            forecasts=full_forecast,
            increment=increment,
            start_time=start_time_utc,
            end_time=end_time_utc
            )

            results = get_recalculating_optimizer_results(
            region=region,
            moer_list = moer_list,
            start_time=start_time_utc,
            end_time=end_time_utc,
            time_needed=time_needed,
            usage_power_kw=usage_power_kw,
            increment=increment
            )
            results_dfs.append(results)
    requery_results = pd.concat(results_dfs)
    
    # baseline + ideal
    moer_list_actuals = prepare_set_of_historic_actuals(
        full_history=full_history,
        start_time=start_time_utc,
        end_time=end_time_utc
        )
    
    no_requery = evu.get_schedule_and_cost_api(
        total_time_horizon=total_intervals_plugged_in,
        usage_power_kw=usage_power_kw,
        time_needed=time_needed,
        optimization_method="auto",
        moer_data=moer_list[0],
        charge_per_interval=None
    )
    no_requery["requery_increment"] = "0"
    
    ideal = evu.get_schedule_and_cost_api(
            total_time_horizon = total_intervals_plugged_in,
            usage_power_kw=usage_power_kw,
            time_needed=time_needed,
            optimization_method="auto",
            moer_data=moer_list_actuals,
            charge_per_interval=None
            )
    ideal["requery_increment"] = "ideal"

    baseline = evu.get_schedule_and_cost_api(
            total_time_horizon = total_intervals_plugged_in,
            usage_power_kw=usage_power_kw,
            time_needed=time_needed,
            optimization_method="baseline",
            moer_data=moer_list_actuals,
            charge_per_interval=None
            )
    baseline["requery_increment"] = "baseline"

    complete_session_cycle = pd.concat([requery_results,baseline,ideal,no_requery]).merge(moer_list_actuals, on="point_time", how="left")
    complete_session_cycle["emissions_co2e_lb_actual"] = complete_session_cycle["value"]*complete_session_cycle["energy_usage_mwh"]
    complete_session_cycle["user_type"] = user_type
    complete_session_cycle["uuid"] = uuid
    all_synth_users_list.append(complete_session_cycle)
region_users_df = pd.concat(all_synth_users_list)
s3.store_csvdataframe(region_users_df,f"results/analysis_requery_20241212_v3_{region}.csv")
print(f"File stored")

144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Sophisticated fit! ==
144
== Sophisticated fit! ==
combining schedules
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! ==
144
== Simple fit! =

IndexError: single positional indexer is out-of-bounds

In [54]:
uuid

'r22.7375_tc71_avglc29236_sdlc7968_contFalse|2024-02-15 15:05:00+00:00|ERCOT_EASTTX'

In [49]:
uuid

'r22.7375_tc71_avglc29236_sdlc7968_contFalse|2024-02-15 15:05:00+00:00|PJM_CHICAGO'

In [40]:
new_start_time = start_time_utc
for fcst_data in moer_list:
    try:
        new_start_time = pd.Timestamp(fcst_data["point_time"].min())
        wt_opt_rc.get_new_schedule(
                new_start_time=new_start_time,
                new_end_time=end_time_utc,
                curr_fcst_data=fcst_data
                )
    except:
        bad = fcst_data
        print(new_start_time)

144
== Simple fit! ==
2024-02-15 15:05:00+00:00
144
== Simple fit! ==
2024-02-15 15:10:00+00:00
144
== Simple fit! ==
2024-02-15 15:15:00+00:00
144
== Simple fit! ==
2024-02-15 15:20:00+00:00
144
== Simple fit! ==
2024-02-15 15:25:00+00:00
144
== Simple fit! ==
2024-02-15 15:30:00+00:00
144
== Simple fit! ==
2024-02-15 15:35:00+00:00
144
== Simple fit! ==
2024-02-15 15:40:00+00:00
144
== Simple fit! ==
2024-02-15 15:45:00+00:00
144
== Simple fit! ==
2024-02-15 15:50:00+00:00
144
== Simple fit! ==
2024-02-15 15:55:00+00:00
144
== Simple fit! ==
2024-02-15 16:00:00+00:00
144
== Simple fit! ==
2024-02-15 16:05:00+00:00
144
== Simple fit! ==
2024-02-15 16:10:00+00:00
144
== Simple fit! ==
2024-02-15 16:15:00+00:00
144
== Simple fit! ==
2024-02-15 16:20:00+00:00
144
== Simple fit! ==
2024-02-15 16:25:00+00:00
144
== Simple fit! ==
2024-02-15 16:30:00+00:00
144
== Simple fit! ==
2024-02-15 16:35:00+00:00
144
== Simple fit! ==
2024-02-15 16:40:00+00:00
144
== Simple fit! ==
2024-02-15 16:45:0

In [41]:
bad

,point_time,value,generated_at,region
4562064,2024-02-15 21:05:00+00:00,1193.7,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
4562065,2024-02-15 21:10:00+00:00,1193.7,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
4562066,2024-02-15 21:15:00+00:00,1192.5,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
4562067,2024-02-15 21:20:00+00:00,1191.3,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
4562068,2024-02-15 21:25:00+00:00,1190.2,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
...,...,...,...,...
4562203,2024-02-16 08:40:00+00:00,587.3,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
4562204,2024-02-16 08:45:00+00:00,600.7,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
4562205,2024-02-16 08:50:00+00:00,600.8,2024-02-15 21:05:00+00:00,ERCOT_EASTTX
4562206,2024-02-16 08:55:00+00:00,599.0,2024-02-15 21:05:00+00:00,ERCOT_EASTTX


In [32]:
moer_list[-2]

,point_time,value,generated_at,region
4561920,2024-02-15 21:00:00+00:00,1193.4,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
4561921,2024-02-15 21:05:00+00:00,1193.4,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
4561922,2024-02-15 21:10:00+00:00,1194.1,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
4561923,2024-02-15 21:15:00+00:00,1191.2,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
4561924,2024-02-15 21:20:00+00:00,1192.1,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
...,...,...,...,...
4562059,2024-02-16 08:35:00+00:00,587.3,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
4562060,2024-02-16 08:40:00+00:00,587.3,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
4562061,2024-02-16 08:45:00+00:00,600.7,2024-02-15 21:00:00+00:00,ERCOT_EASTTX
4562062,2024-02-16 08:50:00+00:00,600.8,2024-02-15 21:00:00+00:00,ERCOT_EASTTX


In [36]:
bad.loc[bad["point_time"] >= new_start_time]["point_time"].iloc[0]

Timestamp('2024-02-15 21:05:00+0000', tz='UTC')

In [30]:
new_start_time

Timestamp('2024-02-15 21:05:00+0000', tz='UTC')

In [ ]:
wt_opt_rc.get_new_schedule(
                new_start_time=new_start_time,
                new_end_time=end_time_utc,
                curr_fcst_data=bad_dat
                )

In [1]:

import os
PATH = os.getenv("HOME")
os.chdir(f"{PATH}/watttime-python-client-aer-algo")

import pandas as pd
import evaluation.eval_framework as evu
from datetime import datetime

import seaborn as sns
import evaluation.metrics as m
from datetime import timedelta

import random
import math
from watttime import WattTimeForecast, WattTimeHistorical
import data.s3 as s3u
import importlib
import watttime.api as wt

import warnings
warnings.filterwarnings("ignore")

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

s3 = s3u.s3_utils()

sample_size = 1000

df_req = s3.load_csvdataframe("requery_data/20241203_1k_synth_users_96_days.csv")

def sanitize_time_needed(x,y):
    return int(math.ceil(min(x, y) / 300.0) * 5)

def sanitize_total_intervals(x):
    return math.ceil(x)

def load_forecast_file(region):
    full_forecast = s3.load_parquetdataframe(
    f"complete_2024_forecast_history/{region}.parquet"
    ).drop_duplicates()

    full_forecast["point_time"] = pd.to_datetime(
    full_forecast["point_time"], utc=True
    )

    return full_forecast

def load_history_file(region):
    return s3.load_parquetdataframe(f"complete_2024_actual_history/{region}.parquet").drop_duplicates()

def prepare_set_of_historic_actuals(
        full_history,
        start_time,
        end_time
        ):

        moer_list = full_history.loc[
                (full_history["point_time"] >= start_time - timedelta(minutes=5)) &
                (full_history["point_time"] <= end_time - timedelta(minutes=5))
                ]
        return moer_list

def prepare_set_of_forecasts(
                forecasts,
                start_time,
                end_time,
                increment
    ):
        inc_times = pd.date_range(
            start_time,
            end_time,
            freq=timedelta(minutes=increment),
        ).tolist()

        moer_list = [
            forecasts.loc[
            forecasts["generated_at"] == timestamp].sort_values(by=["point_time"], ascending=True)
            for timestamp in inc_times
        ]

        return moer_list

def get_recalculating_optimizer_results(
    region: str,
    moer_list: pd.DataFrame,
    start_time: datetime,
    end_time: datetime,
    usage_power_kw,
    time_needed,
    increment,
    charge_per_interval = None,
):
    
    if charge_per_interval is None:
        wt_opt_rc = wt.RecalculatingWattTimeOptimizer(
            region=region,
            watttime_username=username,
            watttime_password=password,
            usage_time_required_minutes=time_needed,
            usage_power_kw=usage_power_kw,
            optimization_method="auto"
            )
    else:
        wt_opt_rc = wt.RecalculatingWattTimeOptimizerWithContiguity(
            username,
            password,
            region,
            usage_time_required_minutes=time_needed,
            usage_power_kw=usage_power_kw,
            optimization_method='auto',
            charge_per_interval=charge_per_interval,
        )

    new_start_time = start_time
    for fcst_data in moer_list:
        new_start_time = pd.Timestamp(fcst_data["point_time"].min())
        wt_opt_rc.get_new_schedule(
                new_start_time=new_start_time,
                new_end_time=end_time,
                curr_fcst_data=fcst_data
                )
    print("combining schedules")
    usage_plan = wt_opt_rc.get_combined_schedule(end_time=end_time)
    usage_plan["requery_increment"] = increment

    return usage_plan

df_req["sanitize_intervals_plugged_in"] = df_req.apply(lambda x: sanitize_total_intervals(x.total_intervals_plugged_in), axis=1)
df_req["sanitize_time_needed"] = df_req.apply(lambda x: sanitize_time_needed(x.total_seconds_to_95, x.length_of_session_in_seconds), axis=1)

synth_data = df_req.sample(sample_size, random_state=42).copy()
synth_data.session_start_time = pd.to_datetime(synth_data.session_start_time)
synth_data.session_end_time = pd.to_datetime(synth_data.session_end_time)

original_regions = [
    "SPP_TX",
    "ERCOT_EASTTX",
    "FPL",
    "SOCO",
    "PJM_CHICAGO",
    "LDWP",
    "PJM_DC",
    "NYISO_NYC",
]

random_regions = [
    'PACE',
    'PNM',
    'MISO_INDIANAPOLIS',
    'WALC',
    'ERCOT_AUSTIN',
    'SPP_KANSAS',
    'ISONE_VT',
    'SPP_SIOUX',
    'SC'
]

regions = original_regions+random_regions

for region in regions[:2]:
    print(region)

    full_forecast = load_forecast_file(region)
    full_history = load_history_file(region)

    all_synth_users_list = []
    bad_dat = []
    increments = [5,15,30,60,90,120,180,240]
    for i in range(0,synth_data.shape[0]):
        try:
            loc_num = i
            time_zone = evu.get_timezone_from_dict(region)                    
            start_time_utc = pd.Timestamp(evu.convert_to_utc(synth_data.iloc[loc_num]['session_start_time'].round('5min') , time_zone))
            end_time_utc = pd.Timestamp(evu.convert_to_utc(synth_data.iloc[loc_num]['session_end_time'].round('5min'), time_zone))
            time_needed = synth_data.iloc[loc_num]["sanitize_time_needed"]
            total_intervals_plugged_in = synth_data.iloc[loc_num]["sanitize_intervals_plugged_in"]
            usage_power_kw = float(synth_data.iloc[loc_num]["power_output_rate"])
            user_type = synth_data.iloc[loc_num]["user_type"]
            optimization_method = "auto"
            uuid = user_type + "|" + str(start_time_utc) + "|" + region
            
            results_dfs = []
            for increment in increments:
                    moer_list = prepare_set_of_forecasts(
                    forecasts=full_forecast,
                    increment=increment,
                    start_time=start_time_utc,
                    end_time=end_time_utc
                    )
                    try:
                        results = get_recalculating_optimizer_results(
                        region=region,
                        moer_list = moer_list,
                        start_time=start_time_utc,
                        end_time=end_time_utc,
                        time_needed=time_needed,
                        usage_power_kw=usage_power_kw,
                        increment=increment
                        )
                        results_dfs.append(results)
                    except:
                        no_go = synth_data.iloc[i].copy(deep=True)
                        no_go["uuid"] = uuid
                        bad_dat.append(no_go)
                        continue
            requery_results = pd.concat(results_dfs)
            
            # baseline + ideal
            moer_list_actuals = prepare_set_of_historic_actuals(
                full_history=full_history,
                start_time=start_time_utc,
                end_time=end_time_utc
                )
            
            no_requery = evu.get_schedule_and_cost_api(
                total_time_horizon=total_intervals_plugged_in,
                usage_power_kw=usage_power_kw,
                time_needed=time_needed,
                optimization_method="auto",
                moer_data=moer_list[0],
                charge_per_interval=None
            )
            no_requery["requery_increment"] = "0"
            
            ideal = evu.get_schedule_and_cost_api(
                    total_time_horizon = total_intervals_plugged_in,
                    usage_power_kw=usage_power_kw,
                    time_needed=time_needed,
                    optimization_method="auto",
                    moer_data=moer_list_actuals,
                    charge_per_interval=None
                    )
            ideal["requery_increment"] = "ideal"

            baseline = evu.get_schedule_and_cost_api(
                    total_time_horizon = total_intervals_plugged_in,
                    usage_power_kw=usage_power_kw,
                    time_needed=time_needed,
                    optimization_method="baseline",
                    moer_data=moer_list_actuals,
                    charge_per_interval=None
                    )
            baseline["requery_increment"] = "baseline"

            complete_session_cycle = pd.concat([requery_results,baseline,ideal,no_requery]).merge(moer_list_actuals, on="point_time", how="left")
            complete_session_cycle["emissions_co2e_lb_actual"] = complete_session_cycle["value"]*complete_session_cycle["energy_usage_mwh"]
            complete_session_cycle["user_type"] = user_type
            complete_session_cycle["uuid"] = uuid
            all_synth_users_list.append(complete_session_cycle)
        except:
            print("bad data")
            continue
    region_users_df = pd.concat(all_synth_users_list)
    s3.store_csvdataframe(region_users_df,f"results/analysis_requery_20241212_v3_{region}.csv")
    print(f"File stored")

if len(bad_dat) > 0:
    bad_dat_df = pd.concat(bad_dat, axis=1).T
    s3.store_csvdataframe(bad_dat_df,f"results/bad_dat_20241212_{region}.csv")

SPP_TX
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Sophisticated fit! ==
== Sophisticated fit! ==
combining schedules
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Simple fit! ==
== Sophisticated fit! ==
combining schedules
== Simple fit! ==
== Simple fit!